## Практика по SQL

### Знакомство с данными

Мы будем работать с набором данных «Книги». Он включает две таблицы, хранящиеся в схеме other, — `other.books` и `other.book_orders`.

<img src=sql_img1.png>

### Задачи

`Задание 09-1`

Составьте список книжных новинок. Новинками считаются все книги за последние пять лет.

Необходимые данные:

* название книги;
* год издания;
* автор;
* жанр.

Вывод отсортируйте по названиям книг.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
order by 1

Посмотрим на полученные данные.

Что-то подозрительно много книг — для рекламного буклета это явно перебор. Что будем делать? Потихоньку избавляться от лишнего.

Скажем, там есть книги без названия? Давайте их исключим из списка.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
order by 1

Теперь нам надо как-то урезать количество оставшихся книг.

Может, по рейтингу автора? Неплохо, только мы не знаем, какие категории авторов у нас есть. Давайте выясним это.

Выберите значения рейтинга авторов, имеющиеся в нашей базе. Отсортируйте вывод по алфавиту.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre,
    author_rating
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
order by 1, 5

Возьмём для рекламного буклета только книги отличных авторов!

Оставьте в выборке новых книг только авторов с рейтингом `'Excellent'`.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre,
    author_rating
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
and (author_rating = 'Excellent' or author_rating = 'Famous')
order by 1